In [7]:
import pandas as pd
import numpy as np
import json
import cloudscraper
import time
import ast
import dropbox

In [19]:
#dropbox token
dbx.files_delete_v2(r'/Levana/egg_nfts.csv')


#https://www.dropbox.com/s/ngnrkibfu1ggm1z/egg_nfts.csv?dl=0

DeleteResult(metadata=FileMetadata(client_modified=datetime.datetime(2022, 1, 25, 20, 36, 28), content_hash='0772dfa1d7aa82c2d05bda5d8f732c268fbb97b9e951aaeeeda5a1a19408c5e1', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:vkl69eWUKy4AAAAAAAAABw', is_downloadable=True, media_info=NOT_SET, name='egg_nfts.csv', parent_shared_folder_id=NOT_SET, path_display='/Levana/egg_nfts.csv', path_lower='/levana/egg_nfts.csv', property_groups=NOT_SET, rev='5d66e0a2525c98e05f701', server_modified=datetime.datetime(2022, 1, 25, 20, 36, 28), sharing_info=NOT_SET, size=3302729, symlink_info=NOT_SET))

In [25]:
with open("egg_nfts.csv", "rb") as f:
    dbx.files_upload(f.read(), '/Levana/egg_nfts.csv', mute = True)

#dbx.files_upload('egg_nfts.csv', r'/Levana/egg_nfts.csv')

In [1]:
def scrape_new(collection_address, csv_name):
    scraper = cloudscraper.create_scraper()
    max_page = json.loads(scraper.get(f"https://randomearth.io/api/items?collection_addr={collection_address}&page=99999").text)['pages']
    for page in range(1, max_page+1):
        counter = 0
        while counter != 1:
            try:
                page_data = scraper.get(f"https://randomearth.io/api/items?collection_addr={collection_address}&page={page}").text
                

                final_df = pd.concat([pd.read_csv(f'{csv_name}.csv'), pd.DataFrame.from_dict(json.loads(page_data)['items'])])
                final_df.to_csv(f'{csv_name}.csv', index=False)
                time.sleep(.30) #yeah yeah it's not efficient, sue me!
                counter += 1
            except Exception as e:
                print(page)
                time.sleep(3)
                pass

In [2]:
collections = {
    'terra1k0y373yxqne22pc9g7jvnr4qclpsxtafevtrpg':'egg_nfts',
    'terra1vhuyuwwr4rkdpez5f5lmuqavut28h5dt29rpn6':'nested_egg_nfts',
    'terra14gfnxnwl0yz6njzet4n33erq5n70wt79nm24el':'loot_nfts',
    'terra1chrdxaef0y2feynkpq63mve0sqeg09acjnp55v':'meteor_nfts',
    'terra1p70x7jkqhf37qa7qm4v23g4u4g8ka4ktxudxa7':'meteor_dust_nfts'
    }

In [ ]:
for address, file_name in collections.items():
    new_df = pd.read_csv('headers.csv')
    new_df.to_csv(f'{file_name}.csv', index=False)
    scrape_new(address, file_name)
    
    dbx.files_delete_v2(fr'/Levana/{file_name}.csv')
